$$dS(t) = \mu S(t) dt + \sigma S(t) dW(t)$$
$$S(T)=S(0)e^{(\mu - \frac{\sigma^2}{2})T + \sigma W(T)}$$
$$W(T) \sim N(0, T)$$

$$S(T_i)=S(T_{i-1})e^{(\mu - \frac{\sigma^2}{2})(T_i - T_{i-1}) + \sigma \sqrt{T_i - T_{i-1}}N(0,1)}$$




In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [ ]:
n_stocks = 5
dt = 1/252
n_years = 1.0
n_observations = int(n_years / dt)

s0 = np.array([100.0, 42.0, 0.5, 24.0, 33333.0])
sigma = np.array([0.2, 0.05, 0.35, 0.4, 1.2])
mu = np.array([0.10, 0.08, 0.12, 0.2, 0.35])

corr_matrix = np.zeros((n_stocks, n_stocks))
trilower_indices = np.tril_indices_from(corr_matrix, k=-1)
corr_matrix[trilower_indices] = np.random.randint(-10, 50, (n_stocks*(n_stocks+1)//2 - n_stocks)) / 100
corr_matrix = corr_matrix + corr_matrix.T
corr_matrix = corr_matrix + np.eye(n_stocks)

mn = multivariate_normal(mean=np.zeros_like(mu), cov=corr_matrix)

sT = np.zeros((n_observations + 1, n_stocks))

sT[0, :] = s0

epsilon = mn.rvs(n_observations)

wT = sigma*np.sqrt(dt)*epsilon

drift = np.ones((n_observations, n_stocks)) * (mu - 0.5*sigma*sigma) * dt

sT[1:, :] = np.exp(drift + wT)
sT_paths = np.cumprod(sT, axis=0)

In [ ]:
print(corr_matrix)

[[ 1.   -0.07  0.45  0.41  0.3 ]
 [-0.07  1.    0.14  0.26  0.44]
 [ 0.45  0.14  1.    0.03  0.16]
 [ 0.41  0.26  0.03  1.    0.41]
 [ 0.3   0.44  0.16  0.41  1.  ]]


In [ ]:
stocks_dataframe = pd.DataFrame(sT_paths)

In [ ]:
stocks_dataframe.columns = ['stock' + str(i) for i in range(n_stocks)]

In [ ]:
stocks_dataframe.describe()

,stock0,stock1,stock2,stock3,stock4
count,253.000000,253.000000,253.000000,253.000000,253.000000
mean,94.667699,43.714746,0.586259,21.280765,9860.653720
std,4.188777,0.958894,0.074508,2.533479,6576.796464
min,87.071810,41.738092,0.424501,16.358958,2089.979070
25%,91.771769,43.238164,0.532653,19.083976,4918.874847
50%,94.312541,43.746754,0.602131,21.243146,8583.301040
75%,96.735055,44.524096,0.632522,23.484850,12814.412696
max,106.488282,45.339140,0.755762,26.641905,38385.278934


In [ ]:
log_returns_dataframe = np.log(stocks_dataframe).diff() * 100

In [ ]:
log_returns_dataframe

,stock0,stock1,stock2,stock3,stock4
0,NaN,NaN,NaN,NaN,NaN
1,0.532710,-0.036360,-4.089432,3.834158,3.658291
2,-0.966627,0.334452,-1.098106,1.327427,10.454322
3,0.891852,-0.246464,-2.163595,-0.529885,-5.167613
4,-2.235988,-0.422810,-2.833570,-0.437752,-19.239848
...,...,...,...,...,...
248,0.201689,0.287046,-0.109999,1.371741,-2.214995
249,0.490071,-0.526625,1.000479,-0.930386,-20.744582
250,0.342773,-0.020862,1.104748,-2.204261,-3.606819
251,0.844157,0.386102,0.806520,5.577787,1.268492


Portfolio relative weights
$$\mathbf{w} \in \mathcal{R}^n$$
Stocks Returns
$$\mathbf{r} \in \mathcal{R}^n$$
My portfolio is made of 300,000\$
$$w_i = \frac{weight_i}{300,000}$$

$$\sum_{i=1}^n w_i = 1$$
$$w_i \geq 0, ∀i$$

Portfolio return
$$\sum_{i=1}^n w_i R_i = R_{ptf} = \mathbf{w} \cdot \mathbf{r}$$


Portfolio variance

$$Var_{ptf} = \mathbf{w}^T \cdot \mathbf{\Sigma} \cdot \mathbf{w}$$

$$\mathbf{\Sigma} \in \mathcal{R}^{n × n}$$
$$\mathbf{w} \in \mathcal{R}^{n \times 1}$$
$$\mathbf{w}^T \in \mathcal{R}^{1 \times n}$$


Minimum variance portfolio
$$ \min_{\mathbf{w}} Var_{ptf}(\mathbf{w}; \mathbf{\Sigma})$$
$$ \sum_{i=1}^n w_i R_i = R^* $$
$$ \sum_{i=1}^n w_i = 1 $$
$$ w_i \geq 0 ∀ i $$

$$ \sum_{i=1}^n w_i R_i - R^* = 0 $$
$$ \sum_{i=1}^n w_i - 1 = 0 $$



In [ ]:
weights = np.ones((n_stocks, )) * 1 / n_stocks

In [ ]:
def ptf_variance(w, covariance_matrix):
    w_column = np.atleast_2d(w).T
    w_T = w_column.T
    w_Sigma = np.dot(w_T, covariance_matrix)
    variance = np.dot(w_Sigma, w_column)[0]
    return variance

def ptf_return(w, r):
    return np.dot(w, r)

In [ ]:
def objective_function(w, covariance_matrix):
    return ptf_variance(w, covariance_matrix)

def expected_returns_constraint_func(w, stock_returns, expected_return):
    return np.dot(w, stock_returns) - expected_return

def total_wealth_constraint_func(w):
    return np.sum(w) - 1.0

In [ ]:
returns = (log_returns_dataframe.mean() / dt).values
Sigma = (log_returns_dataframe.cov() / dt).values

In [ ]:
import scipy.optimize as opt

In [ ]:
def efficient_portfolio(expected_return, market_returns, covariance_matrix):
    expected_return_constraint = {
    'type' : 'eq',
    'fun' : expected_returns_constraint_func,
    'args' : (market_returns, expected_return)
    }
    total_wealth_constraint = { 
        'type' : 'eq',
        'fun' : total_wealth_constraint_func,
    }
    constraints_list = [expected_return_constraint, total_wealth_constraint]
    bounds_list = [ (0.0, None) for _ in market_returns ]

    initial_weights = np.ones_like(market_returns) / len(market_returns)

    solution = opt.minimize(objective_function, initial_weights, method='SLSQP',
                            args=(covariance_matrix, ), bounds=bounds_list, 
                            constraints=constraints_list)
    return solution.x

In [ ]:
expected_returns = [1.0, 3.0, 5.0, 10.0, 25.0, 35.0]
optimal_portfolios = []
for expected_return in expected_returns:
    optimal_weights = efficient_portfolio(expected_return, returns, Sigma)
    optimal_portfolios.append(optimal_weights)

In [ ]:
optimal_portfolios_stdevs = []
optimal_portfolios_returns = []
for w in optimal_portfolios:
    var = ptf_variance(w, Sigma)
    ret = ptf_return(w, returns)
    optimal_portfolios_stdevs.append(np.sqrt(var))
    optimal_portfolios_returns.append(ret)

In [ ]:
optimal_portfolios_stdevs

[array([6.26775882]),
 array([5.71323912]),
 array([5.21103637]),
 array([5.81010535]),
 array([17.82672121]),
 array([27.10243366])]

In [ ]:
optimal_portfolios_returns

[1.000000000560962,
 3.0000000003172502,
 4.999999999744726,
 9.999999955603556,
 24.999994742170003,
 34.99999830117054]